In [1]:
from vizdoom import *
import random
import time
import numpy as np

In [2]:
game = DoomGame()
game.load_config('github/VizDoom/scenarios/basic.cfg')
game.init()

In [3]:
actions = np.identity(3, dtype=np.uint8)

In [7]:
episodes = 10
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        info = state.game_variables
        reward = game.make_action(random.choice(actions))
        print("reward: ", reward)
        time.sleep(0.02)
    print("Result: ", game.get_total_reward())
    time.sleep(2)

reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  100.0
reward:  -1.0
Result:  94.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -6.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -6.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -6.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -6.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
reward:  -1.0
rewar

KeyboardInterrupt: 

In [8]:
game.close()

In [3]:
from gym import Env
from gym.spaces import Discrete, Box
import cv2

In [4]:
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config('github/VizDoom/scenarios/basic.cfg')
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(3)
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [48]:
env = VizDoomGym(render=True)

In [28]:
state = env.reset()

In [50]:
env.close()

In [5]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [49]:
env_checker.check_env(env)

In [51]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        
        return True

In [52]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [53]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [6]:
from stable_baselines3 import PPO

In [55]:
env = VizDoomGym()

In [61]:
model = PPO('CnnPolicy', env, verbose=1, learning_rate=0.0001, n_steps=2048)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [62]:
model.learn(total_timesteps=100000, callback=callback)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.3     |
|    ep_rew_mean     | -97.9    |
| time/              |          |
|    fps             | 26       |
|    iterations      | 1        |
|    time_elapsed    | 78       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 23.3         |
|    ep_rew_mean          | -26.8        |
| time/                   |              |
|    fps                  | 26           |
|    iterations           | 2            |
|    time_elapsed         | 157          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0062449095 |
|    clip_fraction        | 0.144        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | 0.000143     |
|    learning_r

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
new_model = PPO.load('./train/train_basic/best_model_10000')

In [9]:
env = VizDoomGym(render=True)

In [69]:
mean_reward, _ = evaluate_policy(new_model, env, n_eval_episodes=100)

KeyboardInterrupt: 

In [14]:
for episode in range(5):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = new_model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.05)
        total_reward += reward
    print('Total reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total reward for episode 0 is 95.0
Total reward for episode 1 is 51.0
Total reward for episode 2 is 95.0
Total reward for episode 3 is 95.0
Total reward for episode 4 is 95.0
